<a href="https://colab.research.google.com/github/bardiaHSZD/ComputerVisionVFX/blob/main/blender_render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Version of blender used to render the scene.
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/syn73/blender-colab) for more info.

In [1]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install python-opengl -y

!apt install xvfb -y

!pip install pyvirtualdisplay

!pip install piglet


from pyvirtualdisplay import Display
Display().start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 1s (432 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 784 kB of 

In [74]:
blender_version = "3.0.0" #@param ["2.79b", "2.80rc3", "2.81a", "2.82a", "2.83.18", "2.90.1", "2.91.2", "2.92.0", "2.93.5", "3.0.0"]
blend_file_path = 'path/to/file.blend' #@param {type: 'string'}
#@markdown ---
upload_type = 'direct' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'path/to/blend.zip' #@param {type: 'string'}
url_blend = '' #@param {type: 'string'}
#@markdown ---
animation = True #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  210#@param {type: 'integer'}
#@markdown ---
download_type = 'direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}

In [11]:
import os
import shutil
from google.colab import files, drive
uploaded_filename = ""

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    shutil.copy('/drive/My Drive/' + drive_path, '.')
    uploaded_filename = os.path.basename(drive_path)

Saving pols_camera.zip to pols_camera.zip


In [12]:
#!rm -r render
#!mkdir render

if upload_type == 'gdrive_relative':
    if not drive_path.endswith('/'):
        drive_path += '/'
    !cp -r '/drive/My Drive/{drive_path}.' 'render/'
elif uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend and .zip can be uploaded.")

Archive:  pols_camera.zip
  inflating: render/pols_camera.blend  


In [13]:
blend_file_path

'path/to/file.blend'

In [14]:
blender_url_dict = {'2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.80rc3' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.80/blender-2.80rc3-linux-glibc217-x86_64.tar.bz2",
                    '2.81a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2",
                    '2.82a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.82/blender-2.82a-linux64.tar.xz",
                    '2.83.18' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.18-linux-x64.tar.xz",
                    '2.90.1'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz",
                    '2.91.2'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz",
                    '2.92.0'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.92/blender-2.92.0-linux64.tar.xz",
                    '2.93.5'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.5-linux-x64.tar.xz",
                    '3.0.0'   : "https://builder.blender.org/download/daily/blender-3.0.0-beta+v30.f749506163b7-linux.x86_64-release.tar.xz"}

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

--2021-11-23 10:21:01--  https://builder.blender.org/download/daily/blender-3.0.0-beta+v30.f749506163b7-linux.x86_64-release.tar.xz
Resolving builder.blender.org (builder.blender.org)... 82.94.213.213
Connecting to builder.blender.org (builder.blender.org)|82.94.213.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186181668 (178M) [application/x-xz]
Saving to: ‘blender-3.0.0-beta+v30.f749506163b7-linux.x86_64-release.tar.xz’

blender-3.0.0-beta+ 100%[===================>] 177.56M   460KB/s    in 7m 14s  

2021-11-23 10:28:15 (419 KB/s) - ‘blender-3.0.0-beta+v30.f749506163b7-linux.x86_64-release.tar.xz’ saved [186181668/186181668]



In [15]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

In [75]:
#!rm -r output
#!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = 'output/' + output_name
else:
    output_path = '/drive/My Drive/' + drive_output_path + output_name

if animation:
    if start_frame == end_frame:
        !sudo ./$blender_version/blender -b '/content/render/pols_camera.blend' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a
    else:
        !sudo ./$blender_version/blender -b '/content/render/pols_camera.blend' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a
else:
    !sudo ./$blender_version/blender -b '/content/render/pols_camera.blend' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame
    #!sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame

Blender 3.0.0 Beta (hash f749506163b7 built 2021-11-23 01:15:35)
Read blend: /content/render/pols_camera.blend
<bpy_struct, CyclesPreferences at 0x7f748918e868>
Device found: CUDA
Activating <bpy_struct, CyclesDeviceSettings("Tesla P100-PCIE-16GB") at 0x7f7489186b88>
Activating <bpy_struct, CyclesDeviceSettings("Tesla P100-PCIE-16GB") at 0x7f7489186c98>
Fra:161 Mem:38.10M (Peak 39.46M) | Time:00:00.29 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Plane
Fra:161 Mem:38.10M (Peak 39.46M) | Time:00:00.29 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Cylinder
Fra:161 Mem:38.15M (Peak 39.46M) | Time:00:00.31 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Cube.001
Fra:161 Mem:38.15M (Peak 39.46M) | Time:00:00.31 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Sphere
Fra:161 Mem:38.19M (Peak 39.46M) | Time:00:00.31 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Cube.002
Fra:161 Mem:38

In [20]:
path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    if download_type == 'direct':
        files.download('output/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/My Drive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/My Drive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download('output/{}'.format(f))
    # Drive, no zip
    else:
        for f in files_folder:
          shutil.copy("/content/output/" + f, '/drive/My Drive/' + drive_output_path + f)
else:
    raise SystemExit("No frames are rendered.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
#!tar -zcvf output.tar.gz /content/output
!tar cvzf - /content/output | split --bytes=500MB - output.tar.gz

tar: Removing leading `/' from member names
/content/output/
/content/output/blender-65.exr
/content/output/blender-149.exr
/content/output/blender-49.exr
/content/output/blender-58.exr
/content/output/blender-30.exr
/content/output/blender-15.exr
/content/output/blender-05.exr
/content/output/blender-52.exr
/content/output/blender-103.exr
/content/output/blender-98.exr
/content/output/blender-112.exr
/content/output/blender-37.exr
/content/output/blender-24.exr
/content/output/blender-121.exr
/content/output/blender-79.exr
/content/output/blender-87.exr
/content/output/blender-25.exr
/content/output/blender-61.exr
/content/output/blender-18.exr
/content/output/blender-113.exr
/content/output/blender-53.exr
/content/output/blender-47.exr
/content/output/blender-40.exr
/content/output/blender-28.exr
/content/output/blender-145.exr
/content/output/blender-140.exr
/content/output/blender-123.exr
/content/output/blender-06.exr
/content/output/blender-56.exr
/content/output/blender-02.exr
/

In [29]:
!sudo apt update

!sudo apt install ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [828 kB]
Hit:13 http://ppa.laun

In [62]:
!sudo apt update

!sudo apt-get install imagemagick

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubu

In [72]:
!convert /content/output/blender-01.exr -auto-gamma /content/output/blender-01.tiff
!convert /content/output/blender-02.exr -auto-gamma /content/output/blender-02.tiff
!convert /content/output/blender-03.exr -auto-gamma /content/output/blender-03.tiff
!convert /content/output/blender-04.exr -auto-gamma /content/output/blender-04.tiff
!convert /content/output/blender-05.exr -auto-gamma /content/output/blender-05.tiff
!convert /content/output/blender-06.exr -auto-gamma /content/output/blender-06.tiff
!convert /content/output/blender-07.exr -auto-gamma /content/output/blender-07.tiff
!convert /content/output/blender-08.exr -auto-gamma /content/output/blender-08.tiff
!convert /content/output/blender-09.exr -auto-gamma /content/output/blender-09.tiff
!convert /content/output/blender-10.exr -auto-gamma /content/output/blender-10.tiff
!convert /content/output/blender-11.exr -auto-gamma /content/output/blender-11.tiff
!convert /content/output/blender-12.exr -auto-gamma /content/output/blender-12.tiff
!convert /content/output/blender-13.exr -auto-gamma /content/output/blender-13.tiff
!convert /content/output/blender-14.exr -auto-gamma /content/output/blender-14.tiff
!convert /content/output/blender-15.exr -auto-gamma /content/output/blender-15.tiff
!convert /content/output/blender-16.exr -auto-gamma /content/output/blender-16.tiff
!convert /content/output/blender-17.exr -auto-gamma /content/output/blender-17.tiff
!convert /content/output/blender-18.exr -auto-gamma /content/output/blender-18.tiff
!convert /content/output/blender-19.exr -auto-gamma /content/output/blender-19.tiff
!convert /content/output/blender-20.exr -auto-gamma /content/output/blender-20.tiff
!convert /content/output/blender-21.exr -auto-gamma /content/output/blender-21.tiff
!convert /content/output/blender-22.exr -auto-gamma /content/output/blender-22.tiff
!convert /content/output/blender-23.exr -auto-gamma /content/output/blender-23.tiff
!convert /content/output/blender-24.exr -auto-gamma /content/output/blender-24.tiff
!convert /content/output/blender-25.exr -auto-gamma /content/output/blender-25.tiff
!convert /content/output/blender-26.exr -auto-gamma /content/output/blender-26.tiff
!convert /content/output/blender-27.exr -auto-gamma /content/output/blender-27.tiff
!convert /content/output/blender-28.exr -auto-gamma /content/output/blender-28.tiff
!convert /content/output/blender-29.exr -auto-gamma /content/output/blender-29.tiff
!convert /content/output/blender-30.exr -auto-gamma /content/output/blender-30.tiff
!convert /content/output/blender-31.exr -auto-gamma /content/output/blender-31.tiff
!convert /content/output/blender-32.exr -auto-gamma /content/output/blender-32.tiff
!convert /content/output/blender-33.exr -auto-gamma /content/output/blender-33.tiff
!convert /content/output/blender-34.exr -auto-gamma /content/output/blender-34.tiff
!convert /content/output/blender-35.exr -auto-gamma /content/output/blender-35.tiff
!convert /content/output/blender-36.exr -auto-gamma /content/output/blender-36.tiff
!convert /content/output/blender-37.exr -auto-gamma /content/output/blender-37.tiff
!convert /content/output/blender-38.exr -auto-gamma /content/output/blender-38.tiff
!convert /content/output/blender-39.exr -auto-gamma /content/output/blender-39.tiff
!convert /content/output/blender-40.exr -auto-gamma /content/output/blender-40.tiff
!convert /content/output/blender-41.exr -auto-gamma /content/output/blender-41.tiff
!convert /content/output/blender-42.exr -auto-gamma /content/output/blender-42.tiff
!convert /content/output/blender-43.exr -auto-gamma /content/output/blender-43.tiff
!convert /content/output/blender-44.exr -auto-gamma /content/output/blender-44.tiff
!convert /content/output/blender-45.exr -auto-gamma /content/output/blender-45.tiff
!convert /content/output/blender-46.exr -auto-gamma /content/output/blender-46.tiff
!convert /content/output/blender-47.exr -auto-gamma /content/output/blender-47.tiff
!convert /content/output/blender-48.exr -auto-gamma /content/output/blender-48.tiff
!convert /content/output/blender-49.exr -auto-gamma /content/output/blender-49.tiff
!convert /content/output/blender-50.exr -auto-gamma /content/output/blender-50.tiff
!convert /content/output/blender-51.exr -auto-gamma /content/output/blender-51.tiff
!convert /content/output/blender-52.exr -auto-gamma /content/output/blender-52.tiff
!convert /content/output/blender-53.exr -auto-gamma /content/output/blender-53.tiff
!convert /content/output/blender-54.exr -auto-gamma /content/output/blender-54.tiff
!convert /content/output/blender-55.exr -auto-gamma /content/output/blender-55.tiff
!convert /content/output/blender-56.exr -auto-gamma /content/output/blender-56.tiff
!convert /content/output/blender-57.exr -auto-gamma /content/output/blender-57.tiff
!convert /content/output/blender-58.exr -auto-gamma /content/output/blender-58.tiff
!convert /content/output/blender-59.exr -auto-gamma /content/output/blender-59.tiff
!convert /content/output/blender-60.exr -auto-gamma /content/output/blender-60.tiff
!convert /content/output/blender-61.exr -auto-gamma /content/output/blender-61.tiff
!convert /content/output/blender-62.exr -auto-gamma /content/output/blender-62.tiff
!convert /content/output/blender-63.exr -auto-gamma /content/output/blender-63.tiff
!convert /content/output/blender-64.exr -auto-gamma /content/output/blender-64.tiff
!convert /content/output/blender-65.exr -auto-gamma /content/output/blender-65.tiff
!convert /content/output/blender-66.exr -auto-gamma /content/output/blender-66.tiff
!convert /content/output/blender-67.exr -auto-gamma /content/output/blender-67.tiff
!convert /content/output/blender-68.exr -auto-gamma /content/output/blender-68.tiff
!convert /content/output/blender-69.exr -auto-gamma /content/output/blender-69.tiff
!convert /content/output/blender-70.exr -auto-gamma /content/output/blender-70.tiff
!convert /content/output/blender-71.exr -auto-gamma /content/output/blender-71.tiff
!convert /content/output/blender-72.exr -auto-gamma /content/output/blender-72.tiff
!convert /content/output/blender-73.exr -auto-gamma /content/output/blender-73.tiff
!convert /content/output/blender-74.exr -auto-gamma /content/output/blender-74.tiff
!convert /content/output/blender-75.exr -auto-gamma /content/output/blender-75.tiff
!convert /content/output/blender-76.exr -auto-gamma /content/output/blender-76.tiff
!convert /content/output/blender-77.exr -auto-gamma /content/output/blender-77.tiff
!convert /content/output/blender-78.exr -auto-gamma /content/output/blender-78.tiff
!convert /content/output/blender-79.exr -auto-gamma /content/output/blender-79.tiff
!convert /content/output/blender-80.exr -auto-gamma /content/output/blender-80.tiff
!convert /content/output/blender-81.exr -auto-gamma /content/output/blender-81.tiff
!convert /content/output/blender-82.exr -auto-gamma /content/output/blender-82.tiff
!convert /content/output/blender-83.exr -auto-gamma /content/output/blender-83.tiff
!convert /content/output/blender-84.exr -auto-gamma /content/output/blender-84.tiff
!convert /content/output/blender-85.exr -auto-gamma /content/output/blender-85.tiff
!convert /content/output/blender-86.exr -auto-gamma /content/output/blender-86.tiff
!convert /content/output/blender-87.exr -auto-gamma /content/output/blender-87.tiff
!convert /content/output/blender-88.exr -auto-gamma /content/output/blender-88.tiff
!convert /content/output/blender-89.exr -auto-gamma /content/output/blender-89.tiff
!convert /content/output/blender-90.exr -auto-gamma /content/output/blender-90.tiff
!convert /content/output/blender-91.exr -auto-gamma /content/output/blender-91.tiff
!convert /content/output/blender-92.exr -auto-gamma /content/output/blender-92.tiff
!convert /content/output/blender-93.exr -auto-gamma /content/output/blender-93.tiff
!convert /content/output/blender-94.exr -auto-gamma /content/output/blender-94.tiff
!convert /content/output/blender-95.exr -auto-gamma /content/output/blender-95.tiff
!convert /content/output/blender-96.exr -auto-gamma /content/output/blender-96.tiff
!convert /content/output/blender-97.exr -auto-gamma /content/output/blender-97.tiff
!convert /content/output/blender-98.exr -auto-gamma /content/output/blender-98.tiff
!convert /content/output/blender-99.exr -auto-gamma /content/output/blender-99.tiff
!convert /content/output/blender-100.exr -auto-gamma /content/output/blender-100.tiff
!convert /content/output/blender-101.exr -auto-gamma /content/output/blender-101.tiff
!convert /content/output/blender-102.exr -auto-gamma /content/output/blender-102.tiff
!convert /content/output/blender-103.exr -auto-gamma /content/output/blender-103.tiff
!convert /content/output/blender-104.exr -auto-gamma /content/output/blender-104.tiff
!convert /content/output/blender-105.exr -auto-gamma /content/output/blender-105.tiff
!convert /content/output/blender-106.exr -auto-gamma /content/output/blender-106.tiff
!convert /content/output/blender-107.exr -auto-gamma /content/output/blender-107.tiff
!convert /content/output/blender-108.exr -auto-gamma /content/output/blender-108.tiff
!convert /content/output/blender-109.exr -auto-gamma /content/output/blender-109.tiff
!convert /content/output/blender-110.exr -auto-gamma /content/output/blender-110.tiff
!convert /content/output/blender-111.exr -auto-gamma /content/output/blender-111.tiff
!convert /content/output/blender-112.exr -auto-gamma /content/output/blender-112.tiff
!convert /content/output/blender-113.exr -auto-gamma /content/output/blender-113.tiff
!convert /content/output/blender-114.exr -auto-gamma /content/output/blender-114.tiff
!convert /content/output/blender-115.exr -auto-gamma /content/output/blender-115.tiff
!convert /content/output/blender-116.exr -auto-gamma /content/output/blender-116.tiff
!convert /content/output/blender-117.exr -auto-gamma /content/output/blender-117.tiff
!convert /content/output/blender-118.exr -auto-gamma /content/output/blender-118.tiff
!convert /content/output/blender-119.exr -auto-gamma /content/output/blender-119.tiff
!convert /content/output/blender-120.exr -auto-gamma /content/output/blender-120.tiff
!convert /content/output/blender-121.exr -auto-gamma /content/output/blender-121.tiff
!convert /content/output/blender-122.exr -auto-gamma /content/output/blender-122.tiff
!convert /content/output/blender-123.exr -auto-gamma /content/output/blender-123.tiff
!convert /content/output/blender-124.exr -auto-gamma /content/output/blender-124.tiff
!convert /content/output/blender-125.exr -auto-gamma /content/output/blender-125.tiff
!convert /content/output/blender-126.exr -auto-gamma /content/output/blender-126.tiff
!convert /content/output/blender-127.exr -auto-gamma /content/output/blender-127.tiff
!convert /content/output/blender-128.exr -auto-gamma /content/output/blender-128.tiff
!convert /content/output/blender-129.exr -auto-gamma /content/output/blender-129.tiff
!convert /content/output/blender-130.exr -auto-gamma /content/output/blender-130.tiff
!convert /content/output/blender-131.exr -auto-gamma /content/output/blender-131.tiff
!convert /content/output/blender-132.exr -auto-gamma /content/output/blender-132.tiff
!convert /content/output/blender-133.exr -auto-gamma /content/output/blender-133.tiff
!convert /content/output/blender-134.exr -auto-gamma /content/output/blender-134.tiff
!convert /content/output/blender-135.exr -auto-gamma /content/output/blender-135.tiff
!convert /content/output/blender-136.exr -auto-gamma /content/output/blender-136.tiff
!convert /content/output/blender-137.exr -auto-gamma /content/output/blender-137.tiff
!convert /content/output/blender-138.exr -auto-gamma /content/output/blender-138.tiff
!convert /content/output/blender-139.exr -auto-gamma /content/output/blender-139.tiff
!convert /content/output/blender-140.exr -auto-gamma /content/output/blender-140.tiff
!convert /content/output/blender-141.exr -auto-gamma /content/output/blender-141.tiff
!convert /content/output/blender-142.exr -auto-gamma /content/output/blender-142.tiff
!convert /content/output/blender-143.exr -auto-gamma /content/output/blender-143.tiff
!convert /content/output/blender-144.exr -auto-gamma /content/output/blender-144.tiff
!convert /content/output/blender-145.exr -auto-gamma /content/output/blender-145.tiff
!convert /content/output/blender-146.exr -auto-gamma /content/output/blender-146.tiff
!convert /content/output/blender-147.exr -auto-gamma /content/output/blender-147.tiff
!convert /content/output/blender-148.exr -auto-gamma /content/output/blender-148.tiff
!convert /content/output/blender-149.exr -auto-gamma /content/output/blender-149.tiff
!convert /content/output/blender-150.exr -auto-gamma /content/output/blender-150.tiff
!convert /content/output/blender-151.exr -auto-gamma /content/output/blender-151.tiff
!convert /content/output/blender-152.exr -auto-gamma /content/output/blender-152.tiff
!convert /content/output/blender-153.exr -auto-gamma /content/output/blender-153.tiff
!convert /content/output/blender-154.exr -auto-gamma /content/output/blender-154.tiff
!convert /content/output/blender-155.exr -auto-gamma /content/output/blender-155.tiff
!convert /content/output/blender-156.exr -auto-gamma /content/output/blender-156.tiff
!convert /content/output/blender-157.exr -auto-gamma /content/output/blender-157.tiff
!convert /content/output/blender-158.exr -auto-gamma /content/output/blender-158.tiff
!convert /content/output/blender-159.exr -auto-gamma /content/output/blender-159.tiff
!convert /content/output/blender-160.exr -auto-gamma /content/output/blender-160.tiff

In [73]:
!ffmpeg -start_number 1 -i /content/output/blender-%02d.tiff -vcodec mpeg4 /content/output/test.avi

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

## Disclaimer
The GPU used in Google Colab is specialized for data centres, neural network etc, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies. [ColabPro](https://colab.research.google.com/signup) is available for those who wanted to have more powerful GPU and longer session for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info about this platform. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020 syn73

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```